In [95]:
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# SQL statements to drop tables if they exist
ddl_drop = [
    """DROP TABLE IF EXISTS "retired_value";""",
    """DROP TABLE IF EXISTS "value";""",
    """DROP TABLE IF EXISTS "statement";""",
    """DROP TABLE IF EXISTS "property";""",
    """DROP TABLE IF EXISTS "record";""",
    """DROP TABLE IF EXISTS "transaction";""",
    """DROP TABLE IF EXISTS "registry";"""
]

# Execute each DROP TABLE SQL statement
for sql in ddl_drop:
    cursor.execute(sql)

# SQL statements to create new tables
create_ddl = [
    """
    CREATE TABLE IF NOT EXISTS "registry" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "transaction" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "registry_id" INTEGER,
        FOREIGN KEY ("registry_id") REFERENCES "registry"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "record" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "registry_id" INTEGER,
        "identifier" TEXT,
        FOREIGN KEY ("registry_id") REFERENCES "registry"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "property" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "type_name" TEXT,
        "property_name" TEXT,
        "multi" INTEGER DEFAULT 'FALSE'
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "statement" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "record_id" INTEGER,
        "property_id" INTEGER,
        "transaction_id" INTEGER,
        "effective_from" DATETIME,
        "created" DATETIME,
        FOREIGN KEY ("record_id") REFERENCES "record"("id"),
        FOREIGN KEY ("property_id") REFERENCES "property"("id"),
        FOREIGN KEY ("transaction_id") REFERENCES "transaction"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "value" (
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "statement_id" INTEGER,
        "payload" TEXT,
        FOREIGN KEY ("statement_id") REFERENCES "statement"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "retired_value" (
        "id" INTEGER PRIMARY KEY,
        FOREIGN KEY ("id") REFERENCES "value"("id")
    );
    """
]

# Execute each CREATE TABLE SQL statement
for sql in create_ddl:
    cursor.execute(sql)

# Commit the changes and close the database connection
connection.commit()
connection.close()


In [96]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Transaction 1 - 3 statements
seed_dml = [
    """INSERT INTO "registry" ("id") VALUES (1);""",
    """INSERT INTO "transaction" ("registry_id") VALUES (1);""",
    """INSERT INTO "record" ("registry_id", "identifier") VALUES (1, '90110800000');""",
    """
    INSERT INTO "property" ("type_name", "property_name", "multi")
    VALUES 
    ('Person', 'firstName', 'TRUE'),
    ('Person', 'gender', 'FALSE'),
    ('Person', 'citizenship', 'TRUE'),
    ('Residency', 'residence', 'TRUE');
    """,
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES
    (1, 1, 1, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id", "payload")
    VALUES
    (1, 'John');
    """,
    """
    INSERT INTO "value" ("statement_id", "payload")
    VALUES
    (1, 'Fitzgerald');
    """,
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES
    (1, 2, 1, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id", "payload")
    VALUES
    (2, 'Male');
    """,
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES 
    (1, 3, 1, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id", "payload")
    VALUES
    (3, '{"isoCode": "BEL", "nisCode": "150"}');
    """,
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES 
    (1, 4, 1, '2020-01-01 10:00:00', '2020-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id", "payload")
    VALUES 
    (4, '{"address": {"bestIdentifier": {"id": "https://data.vlaanderen.be/id/adres/3706808", "namespace": "https://data.vlaanderen.be/id/adres", "objectId": "3706808", "versionId": "2023-11-01T15:46:20+01:00"}}}');
    """,
]

for statement in seed_dml:
    cursor.execute(statement)

connection.commit()
connection.close()

In [108]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Select all statements.
query = """
SELECT s.id AS statement_id, r.identifier AS record_identifier, p.type_name, p.property_name, v.payload AS value, IIF(rv.id, 1, 0) AS retired_value, s.effective_from, s.created 
FROM "statement" s
JOIN "record" r ON r.id = s.record_id
JOIN "property" p ON s.property_id = p.id
JOIN "value" v ON s.id = v.statement_id
LEFT JOIN "retired_value" rv ON v.id = rv.id
ORDER BY p.type_name, p.property_name;
"""

resultset_df = pd.read_sql(query, connection)
resultset_df["retired_value"] = resultset_df["retired_value"].astype(bool)
 
connection.close()

resultset_df

,statement_id,record_identifier,type_name,property_name,value,retired_value,effective_from,created
0,3,90110800000,Person,citizenship,"{""isoCode"": ""BEL"", ""nisCode"": ""150""}",True,2020-01-01 10:00:00,2020-01-01 10:00:00
1,1,90110800000,Person,firstName,John,False,2020-01-01 10:00:00,2020-01-01 10:00:00
2,1,90110800000,Person,firstName,Fitzgerald,True,2020-01-01 10:00:00,2020-01-01 10:00:00
3,5,90110800000,Person,firstName,None,False,2022-01-01 10:00:00,2022-01-01 10:00:00
4,5,90110800000,Person,firstName,Cornelius,False,2022-01-01 10:00:00,2022-01-01 10:00:00
5,6,90110800000,Person,firstName,None,False,2022-01-01 10:00:00,2022-01-01 10:00:00
6,2,90110800000,Person,gender,Male,False,2020-01-01 10:00:00,2020-01-01 10:00:00
7,4,90110800000,Residency,residence,"{""address"": {""bestIdentifier"": {""id"": ""https:/...",False,2020-01-01 10:00:00,2020-01-01 10:00:00


In [98]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Return current state of record.
query = """
SELECT 
    r.identifier AS record_identifier, p.type_name, p.property_name, v.payload AS value, s.effective_from, s.created
FROM "statement" s
JOIN "record" r ON s.record_id = r.id
JOIN "property" p ON s.property_id = p.id
JOIN "value" v ON  s.id = v.statement_id
LEFT JOIN "retired_value" rv ON v.id = rv.id
WHERE record_id = :record_id
AND rv.id IS NULL
AND v.payload IS NOT NULL
ORDER BY p.type_name, p.property_name;
"""

resultset_df = pd.read_sql(query, connection, params={"record_id": 1})

connection.close()

resultset_df

,record_identifier,type_name,property_name,value,effective_from,created
0,90110800000,Person,citizenship,"{""isoCode"": ""BEL"", ""nisCode"": ""150""}",2020-01-01 10:00:00,2020-01-01 10:00:00
1,90110800000,Person,firstName,John,2020-01-01 10:00:00,2020-01-01 10:00:00
2,90110800000,Person,firstName,Fitzgerald,2020-01-01 10:00:00,2020-01-01 10:00:00
3,90110800000,Person,gender,Male,2020-01-01 10:00:00,2020-01-01 10:00:00
4,90110800000,Residency,residence,"{""address"": {""bestIdentifier"": {""id"": ""https:/...",2020-01-01 10:00:00,2020-01-01 10:00:00


In [99]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Update property
# Transaction 2 - 1 statement 
retract_dml = [
    """INSERT INTO "transaction" ("registry_id") VALUES (1);""",
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES 
    (1, 1, 2, '2022-01-01 10:00:00', '2022-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id")
    VALUES (5)
    """,
    """
    INSERT INTO "retired_value"
    VALUES (2)
    """,
    """
    INSERT INTO "value" ("statement_id", "payload") 
    VALUES (5, 'Cornelius');
    """,
]

for statement in retract_dml:
    cursor.execute(statement)

connection.commit()
connection.close()

In [100]:
import sqlite3

connection = sqlite3.connect("registry.db")
cursor = connection.cursor()

# Retract (soft delete) property
# Transaction 3 - 1 statement 
retract_dml = [
    """INSERT INTO "transaction" ("registry_id") VALUES (1);""",
    """
    INSERT INTO "statement" ("record_id", "property_id", "transaction_id", "effective_from", "created")
    VALUES 
    (1, 1, 3, '2022-01-01 10:00:00', '2022-01-01 10:00:00');
    """,
    """
    INSERT INTO "value" ("statement_id")
    VALUES (6)
    """,
    """
    INSERT INTO "retired_value"
    VALUES (4)
    """,
]

for statement in retract_dml:
    cursor.execute(statement)

connection.commit()
connection.close()